In [1]:
import os
from pathlib import Path
import imageio.v3 as iio
import numpy as np

# ── USER CONFIG ─────────────────────────────────────────────────────────
# List all of the “source” folders you want to convert:
SOURCE_FOLDERS = [
    Path("../davis2017eval/sam2_preds_MA_0.15"),
    Path("../davis2017eval/sam2_preds_MA_OF_0.15"),
    Path("../davis2017eval/sam2_preds_Naive_0.15"),
    Path("../davis2017eval/sam2_preds_Naive_OF_0.15"),
    Path("../davis2017eval/sam2_preds_MA_0.05"),
    Path("../davis2017eval/sam2_preds_MA_OF_0.05"),
    Path("../davis2017eval/sam2_preds_Naive_0.05"),
    Path("../davis2017eval/sam2_preds_Naive_OF_0.05"),
    Path("../davis2017eval/sam2_preds_MA_0.07"),
    Path("../davis2017eval/sam2_preds_MA_OF_0.07"),
    Path("../davis2017eval/sam2_preds_Naive_0.07"),
    Path("../davis2017eval/sam2_preds_Naive_OF_0.07"),
    Path("../davis2017eval/sam2_preds_MA_0.10"),
    Path("../davis2017eval/sam2_preds_MA_OF_0.10"),
    Path("../davis2017eval/sam2_preds_Naive_0.10"),
    Path("../davis2017eval/sam2_preds_Naive_OF_0.10"),
]

# (No need for a single TARGET_DIR variable anymore; we’ll generate a "_gray" folder per source.)
# ── HELPER: find the “object‐colors → ID” mapping from frame 0 ─────────────
def extract_color_to_id_map(first_color_png: Path):
    """
    Read the RGB image first_color_png (H×W×3), find all non‐black colors,
    and assign them IDs 1,2,3,… in sorted order. Returns:
      • color2id: dict mapping (R,G,B) tuples → integer ID
      • H, W: image dimensions
    """
    rgb = iio.imread(str(first_color_png))
    if rgb.ndim != 3 or rgb.shape[2] != 3:
        raise RuntimeError(f"Expected a (H, W, 3) image at {first_color_png}")

    flat = rgb.reshape(-1, 3)
    uniq = np.unique(flat, axis=0)       # (K,3) array of all colors present
    # Drop black (0,0,0):
    non_black = [tuple(c) for c in uniq if not np.all(c == 0)]
    if len(non_black) == 0:
        raise RuntimeError(f"No non‐black colors found in {first_color_png}")

    # Sort by RGB lex order (optional) to assign stable IDs:
    non_black.sort()
    color2id = {color: (i + 1) for i, color in enumerate(non_black)}
    return color2id, rgb.shape[0], rgb.shape[1]


# ── MAIN: iterate over each of the source‐root folders in SOURCE_FOLDERS ───
for src_root in SOURCE_FOLDERS:
    if not src_root.exists() or not src_root.is_dir():
        print(f"Warning: '{src_root}' does not exist or is not a directory. Skipping.")
        continue

    # Build a parallel “gray” root by adding "_gray" to the folder name:
    gray_root = src_root.with_name(src_root.name + "_gray")
    gray_root.mkdir(parents=True, exist_ok=True)

    print(f"\n=== Converting entire folder: {src_root} → {gray_root} ===")

    # (Within each source‐root, we still expect multiple per‐sequence subfolders.)
    for seq_folder in sorted(src_root.iterdir()):
        if not seq_folder.is_dir():
            continue

        print(f"  Converting sequence: {seq_folder.name}")
        out_seq = gray_root / seq_folder.name
        out_seq.mkdir(parents=True, exist_ok=True)

        # (1) find frame 00000.png and build the color→ID lookup
        first_frame = seq_folder / "00000.png"
        if not first_frame.exists():
            raise RuntimeError(f"Cannot find {first_frame}")

        color2id, H, W = extract_color_to_id_map(first_frame)
        # e.g. color2id might be {(200, 0, 0): 1, (0, 200, 0): 2}, etc.

        # (2) iterate over all PNGs in this sequence folder (00000.png, 00001.png, …)
        all_frames = sorted(seq_folder.glob("*.png"))
        for frame_path in all_frames:
            rgb = iio.imread(str(frame_path))
            if rgb.ndim != 3 or rgb.shape[:2] != (H, W):
                raise RuntimeError(f"Unexpected image shape in {frame_path}: {rgb.shape}")

            # Build a blank H×W array of uint8
            id_map = np.zeros((H, W), dtype=np.uint8)

            # For each distinct color in color2id, mask and assign ID
            # (Pixels that remain black → ID=0)
            for (R, G, B), obj_id in color2id.items():
                mask = (rgb[:, :, 0] == R) & (rgb[:, :, 1] == G) & (rgb[:, :, 2] == B)
                if mask.any():
                    id_map[mask] = obj_id

            # Save the new single‐channel PNG
            out_path = out_seq / frame_path.name
            iio.imwrite(str(out_path), id_map)

        print(f"  → Saved gray masks in {out_seq}")

    print(f"Finished processing “{src_root.name}”. Gray outputs in “{gray_root}”.")
print("\nAll requested folders converted to grayscale‐ID format.")



=== Converting entire folder: ../davis2017eval/sam2_preds_MA_0.15 → ../davis2017eval/sam2_preds_MA_0.15_gray ===
  Converting sequence: bike-packing
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/bike-packing
  Converting sequence: blackswan
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/blackswan
  Converting sequence: bmx-trees
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/bmx-trees
  Converting sequence: breakdance
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/breakdance
  Converting sequence: camel
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/camel
  Converting sequence: car-roundabout
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/car-roundabout
  Converting sequence: car-shadow
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/car-shadow
  Converting sequence: cows
  → Saved gray masks in ../davis2017eval/sam2_preds_MA_0.15_gray/cows
  Converting sequenc